## Merge all data

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
energy_df = pd.read_csv('energy_data.csv', index_col=0)
new_sp_df = pd.read_csv('sp500_news_data.csv', index_col=0)
gold_df = pd.read_csv('daily_gold_rate.csv')
pattern_df = pd.read_csv('candlestick_patterns.csv', index_col=0)

In [3]:
new_sp_df

,date,Open,High,Low,Close,Volume,up/down,Sentiment,true_value
0,1980-01-02,-1.157349,-1.064447,-1.065108,-1.066003,-0.960949,1.0,-0.632982,-1.0
1,1980-01-03,-1.157349,-1.066807,-1.067169,-1.066548,-0.955609,1.0,-0.547975,1.0
2,1980-01-07,-1.157349,-1.065080,-1.064590,-1.064943,-0.958844,1.0,-0.766564,1.0
3,1980-01-08,-1.157349,-1.063584,-1.064093,-1.062783,-0.954035,1.0,-1.139702,1.0
4,1980-01-09,-1.157349,-1.061777,-1.061940,-1.062682,-0.947613,1.0,-1.267481,1.0
...,...,...,...,...,...,...,...,...,...
10664,2022-05-23,2.769766,2.824832,2.796829,2.837995,0.852602,1.0,-0.469044,-1.0
10665,2022-05-24,2.793333,2.798525,2.762400,2.805425,1.127906,-1.0,-0.533289,1.0
10666,2022-05-25,2.779956,2.842354,2.813063,2.843022,1.355428,1.0,-0.645402,1.0
10667,2022-05-26,2.835074,2.918473,2.873545,2.922868,1.160529,1.0,-0.683530,1.0


In [4]:
energy_df.rename(columns={"Date": "date"},inplace=True)

In [5]:
energy_df

,date,NY_cgas,US_cgas,WTI_crude_oil,NY_hoil
0,1986-06-02,0.468,0.445,13.80,0.402
1,1986-06-03,0.436,0.418,13.35,0.393
2,1986-06-04,0.418,0.398,13.15,0.378
3,1986-06-05,0.431,0.415,13.21,0.390
4,1986-06-06,0.421,0.403,12.73,0.385
...,...,...,...,...,...
9039,2022-05-24,3.849,3.794,112.55,3.911
9040,2022-05-25,3.898,3.843,112.88,3.994
9041,2022-05-26,3.927,3.887,116.19,3.961
9042,2022-05-27,4.102,4.027,114.96,3.902


In [6]:
#only USD
gold_df = gold_df[['Date','USD']]
gold_df

,Date,USD
0,1985-01-01,308.30
1,1985-01-02,305.50
2,1985-01-03,302.30
3,1985-01-04,303.15
4,1985-01-07,298.25
...,...,...
9754,2022-05-23,1856.20
9755,2022-05-24,1867.10
9756,2022-05-25,1847.20
9757,2022-05-26,1848.25


In [7]:
#rename columm
gold_df.rename(columns = {'Date':'date'}, inplace = True)
gold_df

,date,USD
0,1985-01-01,308.30
1,1985-01-02,305.50
2,1985-01-03,302.30
3,1985-01-04,303.15
4,1985-01-07,298.25
...,...,...
9754,2022-05-23,1856.20
9755,2022-05-24,1867.10
9756,2022-05-25,1847.20
9757,2022-05-26,1848.25


In [8]:
from sklearn.preprocessing import StandardScaler

#normalize data using sklearn
normalizer = StandardScaler()
df_dropped = gold_df.drop('date', axis = 1)
normalizer_gold_df = pd.DataFrame(normalizer.fit_transform(df_dropped), columns = df_dropped.columns)
normalizer_gold_df.insert(loc = 0, column = 'date', value = gold_df['date'])
normalizer_gold_df


,date,USD
0,1985-01-01,-0.878967
1,1985-01-02,-0.884399
2,1985-01-03,-0.890607
3,1985-01-04,-0.888958
4,1985-01-07,-0.898463
...,...,...
9754,2022-05-23,2.123904
9755,2022-05-24,2.145050
9756,2022-05-25,2.106444
9757,2022-05-26,2.108481


In [1]:
pattern_df

NameError: name 'pattern_df' is not defined

In [10]:
#merge data
df_names = [energy_df,new_sp_df,normalizer_gold_df,pattern_df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['date'],
                                            how='outer'), df_names)

In [11]:
#drop NAN rows
df_merged = df_merged.dropna().reset_index(drop=True)
df_merged

,date,NY_cgas,US_cgas,WTI_crude_oil,NY_hoil,Open,High,Low,Close,Volume,...,CDLSHOOTINGSTAR_Bear,CDLSPINNINGTOP_Bear,CDLSPINNINGTOP_Bull,CDLSTICKSANDWICH_Bull,CDLTASUKIGAP_Bear,CDLTASUKIGAP_Bull,CDLTHRUSTING_Bear,CDLTRISTAR_Bear,CDLTRISTAR_Bull,NO_PATTERN
0,1986-06-02,0.468,0.445,13.80,0.402,-0.910826,-0.924568,-0.924449,-0.925426,-0.917674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1986-06-03,0.436,0.418,13.35,0.393,-0.911828,-0.926807,-0.924611,-0.924952,-0.920866,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1986-06-04,0.418,0.398,13.15,0.378,-0.911357,-0.926014,-0.925708,-0.926537,-0.919621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1986-06-05,0.431,0.415,13.21,0.390,-0.912930,-0.926657,-0.924875,-0.924811,-0.922921,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1986-06-06,0.421,0.403,12.73,0.385,-0.911217,-0.926245,-0.923840,-0.924791,-0.922921,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8914,2021-12-27,2.283,2.228,75.49,2.240,3.585936,3.637751,3.634398,3.663045,0.241992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8915,2021-12-28,2.293,2.236,76.01,2.253,3.647556,3.653344,3.681153,3.658160,0.216526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8916,2021-12-29,2.317,2.260,76.58,2.259,3.640693,3.650372,3.679163,3.664932,0.298933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8917,2021-12-30,2.322,2.264,76.83,2.263,3.646294,3.655262,3.676371,3.650469,0.310630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
#save dataframe
df_merged.to_csv('cleaned_data.csv')